# install talib

In [1]:
!cp ../input/talibinstall/ta-lib-0.4.0-src.tar.gzh  ./ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz > null
!cd ta-lib && ./configure --prefix=/usr > null && make  > null && make install > null
!cp ../input/talibinstall/TA-Lib-0.4.21.tar.gzh TA-Lib-0.4.21.tar.gz
!pip install TA-Lib-0.4.21.tar.gz > null
!pip install ../input/talibinstall/numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl >null
import talib
talib.__version__

./configure: line 4349: /usr/bin/file: No such file or directory
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
ar: `u' modifier ignored since `D' is the default (see `U')
gen_code.c: In function ‘printFuncHeaderDoc’:
gen_code.c:3456:4: warning: format not a string literal and no format arguments [-Wformat-security]
 3456 |    fprintf( out, prefix );
      |    ^~~~~~~
ERROR: pip's dependency resolver does not currently take into a

'0.4.21'

### メモリ削減

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
import psutil

# 特徴量エンジニアリング

In [32]:
import pandas as pd
import numpy as np
import talib
from sklearn.preprocessing import StandardScaler
import gc
import time
from concurrent import futures

class Feature():

    def __init__(self) -> None:
        pass
    
    def __del__(self):
        """オブジェクトが破棄されるとき呼び出される"""
        print('Feature died:', id(self))
    def MACD(self, close : pd.DataFrame, fastperiod=324, slowperiod=296, signalperiod=272):
        exp1 = close.rolling(fastperiod).mean()
        exp2 = close.rolling(slowperiod).mean()
        macd = 100 * (exp1 - exp2) / exp2
        signal = macd.rolling(signalperiod).mean()
        hist = signal - macd

        return hist
    
    def task(self, num, df):
        if num == 0:
            return df.iloc[-16,7] / df.iloc[-1,7]
        elif num == 1:
            return df.iloc[-46,7] / df.iloc[-1,7]
        elif num == 2:
            return df.iloc[-91,7] / df.iloc[-1,7]
        elif num == 3:
            return df.iloc[-181,7] / df.iloc[-1,7]
        elif num == 4:
            return df.iloc[-211,7] / df.iloc[-1,7]
        elif num == 5:
            return df.iloc[-311,7] / df.iloc[-1,7]
        elif num == 6:
            return df.iloc[-351,7] / df.iloc[-1,7]
        elif num == 7:
            return df.iloc[-451,7] / df.iloc[-1,7]
        elif num == 8:
            return talib.ROCP(df.iloc[-182:,5], timeperiod=181).iloc[-1]
        elif num == 9:
            return talib.MOM(df.iloc[-177:,5], timeperiod=176).iloc[-1]
        elif num == 10:
            return talib.RSI(df.iloc[-207:,5], timeperiod=206).iloc[-1]
        elif num == 11:
            macdhist = self.MACD(df.iloc[-610:,5], fastperiod=324, slowperiod=296, signalperiod=272)
            return macdhist.iloc[-1]
        elif num == 12:
            macdhist = self.MACD(df.iloc[-610:,5], fastperiod=324, slowperiod=296, signalperiod=272)
            return macdhist.iloc[-2]
        elif num == 13:
            macdhist = self.MACD(df.iloc[-610:,5], fastperiod=324, slowperiod=296, signalperiod=272)
            return macdhist.iloc[-6]
        elif num == 14:
            macdhist = self.MACD(df.iloc[-610:,5], fastperiod=324, slowperiod=296, signalperiod=272)
            return macdhist.iloc[-16]
        elif num == 15:
            return talib.LINEARREG_ANGLE(df.iloc[-182:,5], timeperiod=181).iloc[-1]
        elif num == 16:
            return talib.HT_DCPERIOD(df.iloc[-1000:,5]).iloc[-1]
        elif num == 17:
            return talib.AROONOSC(df.iloc[-182:,3], df.iloc[-182:,4], timeperiod=181).iloc[-1]
        elif num == 18:
            aroondown, aroonup = talib.AROON(df.iloc[-182:,3], df.iloc[-182:,4], timeperiod=181)
            return aroondown.iloc[-1]
        elif num == 19:
            aroondown, aroonup = talib.AROON(df.iloc[-182:,3], df.iloc[-182:,4], timeperiod=181)
            return aroonup.iloc[-1]
        elif num == 20:
            return talib.PLUS_DI(df.iloc[-182:,3], df.iloc[-182:,4], df.iloc[-182:,5], timeperiod=181).iloc[-1]
        elif num == 21:
            upper1, middle,lower1 = talib.BBANDS(df.iloc[-182:,5], timeperiod=181, nbdevup=3, nbdevdn=3, matype=0)
            return (upper1.iloc[-2:] - df.iloc[-2:,5]).apply(np.log).iloc[-1]
            

    def conv_data(self, df, Asset_ID, df_list, save_fet=False, save_name='feature', save_mem=True, test=False):
        
        df.fillna(method='ffill', inplace=True)
        
        if not test:
        
            for i in range(13):
                if not (i == Asset_ID and i == 0 and i == 7):
                    as_df = df_list[i].copy()
                    #df[str(i)+'VWAP_shift105'] = as_df['VWAP'].shift(105) / as_df["VWAP"]
                    #df[str(i)+'MOM'] = talib.MOM(as_df['Close'], timeperiod=176)

                    del as_df
                    gc.collect()

            df['VWAP_shift15'] = df['VWAP'].shift(15) / df["VWAP"]
            #df['VWAP_shift30'] = df['VWAP'].shift(30) / df["VWAP"]
            df['VWAP_shift45'] = df['VWAP'].shift(45) / df["VWAP"]
            df['VWAP_shift90'] = df['VWAP'].shift(90) / df["VWAP"]
            #df['VWAP_shift120'] = df['VWAP'].shift(120) / df["VWAP"]
            df['VWAP_shift180'] = df['VWAP'].shift(180) / df["VWAP"]
            df['VWAP_shift210'] = df['VWAP'].shift(210) / df["VWAP"]
            #df['VWAP_shift240'] = df['VWAP'].shift(240) / df["VWAP"]
            df['VWAP_shift310'] = df['VWAP'].shift(310) / df["VWAP"]
            df['VWAP_shift350'] = df['VWAP'].shift(350) / df["VWAP"]
            #df['VWAP_shift400'] = df['VWAP'].shift(400) / df["VWAP"]
            df['VWAP_shift450'] = df['VWAP'].shift(450) / df["VWAP"]
            df['VWAP_shift550'] = df['VWAP'].shift(550) / df["VWAP"]
            df['VWAP_shift600'] = df['VWAP'].shift(600) / df["VWAP"]
            df['VWAP_shift650'] = df['VWAP'].shift(650) / df["VWAP"]
            #df['VWAP_shift750'] = df['VWAP'].shift(750) / df["VWAP"]
            df['VWAP_shift800'] = df['VWAP'].shift(800) / df["VWAP"]
            df['VWAP_shift1000'] = df['VWAP'].shift(1000) / df["VWAP"]
            df['VWAP_shift1500'] = df['VWAP'].shift(1500) / df["VWAP"]
            df['VWAP_shift2000'] = df['VWAP'].shift(2000) / df["VWAP"]
            df['VWAP_shift2500'] = df['VWAP'].shift(2500) / df["VWAP"]
            df['VWAP_shift3000'] = df['VWAP'].shift(3000) / df["VWAP"]
            df['VWAP_shift3500'] = df['VWAP'].shift(3500) / df["VWAP"]
            df['VWAP_shift4000'] = df['VWAP'].shift(4000) / df["VWAP"]
            df['VWAP_shift4500'] = df['VWAP'].shift(4500) / df["VWAP"]
            df['VWAP_shift5000'] = df['VWAP'].shift(5000) / df["VWAP"]
            df['VWAP_shift5500'] = df['VWAP'].shift(5500) / df["VWAP"]
            df['VWAP_shift6000'] = df['VWAP'].shift(6000) / df["VWAP"]
            df['VWAP_shift7000'] = df['VWAP'].shift(7000) / df["VWAP"]
            df['VWAP_shift8000'] = df['VWAP'].shift(8000) / df["VWAP"]
            df['VWAP_shift9000'] = df['VWAP'].shift(9000) / df["VWAP"]
            df['VWAP_shift10000'] = df['VWAP'].shift(10000) / df["VWAP"]
            df['VWAP_shift11000'] = df['VWAP'].shift(11000) / df["VWAP"]
            df['VWAP_shift12000'] = df['VWAP'].shift(12000) / df["VWAP"]
            df['VWAP_shift14000'] = df['VWAP'].shift(14000) / df["VWAP"]
            df['VWAP_shift16000'] = df['VWAP'].shift(16000) / df["VWAP"]
            df['VWAP_shift18000'] = df['VWAP'].shift(18000) / df["VWAP"]
            df['VWAP_shift25000'] = df['VWAP'].shift(25000) / df["VWAP"]
#             df['Volume_shift_sma2000'] = (df['Volume'].rolling(2000).sum() / df["Volume"]).apply(np.log)
#             df['Close_shift70'] = df['Close'].shift(70) / df["Close"]
#             df['Close_shift280'] = df['Close'].shift(280) / df["Close"]
#             df['SMA_18000_std'] = df['Close'].rolling(18000).std().shift() / df['Close']
            df['ROCP'] = talib.ROCP(df['Close'], timeperiod=181)
            df['MOM'] = talib.MOM(df['Close'], timeperiod=176)
#             df['MOM_1700'] = talib.MOM(df['Close'], timeperiod=1760)
            df['RSI'] = talib.RSI(df['Close'], timeperiod=206)
            #df['EMA'] = (talib.EMA(df['Close'], timeperiod=11) - df['Close']) / df["Close"]
#             df['APO'] = talib.APO(df['Close'], fastperiod=117, slowperiod=166, matype=0)
#             df['CMO'] = talib.CMO(df['Close'], timeperiod=204)
            macdhist = self.MACD(df['Close'], fastperiod=324, slowperiod=296, signalperiod=272)
            df['macdhist0'] = macdhist
            df['macdhist1'] = macdhist.shift(1)
            df['macdhist5'] = macdhist.shift(5)
            df['macdhist15'] = macdhist.shift(15)
#             df['ADX'] = talib.ADX(df["High"], df["Low"], df["Close"], timeperiod=181)
            df['LINEARREG_ANGLE'] = talib.LINEARREG_ANGLE(df["Close"], timeperiod=181)
            df['HT_DCPERIOD'] = talib.HT_DCPERIOD(df["Close"])
#             df['NATR'] = talib.NATR(df["High"], df["Low"], df["Close"], timeperiod=181)
            df['AROONOSC'] = talib.AROONOSC(df["High"], df["Low"], timeperiod=181)
            aroondown, aroonup = talib.AROON(df["High"], df["Low"], timeperiod=181)
            df['aroondown'] = aroondown
            df['aroonup'] = aroonup
            df['PLUS_DI'] = talib.PLUS_DI(df["High"], df["Low"], df["Close"], timeperiod=181)
            upper1, middle,lower1 = talib.BBANDS(df["Close"], timeperiod=181, nbdevup=3, nbdevdn=3, matype=0)
            df['BBANDS'] = (upper1 - df['Close']).apply(np.log)
#             df['CDL2CROWS'] = talib.CDL2CROWS(df["Open"], df["High"], df["Low"], df["Close"])

#             categorical_features = ['CDL2CROWS']
            
            delete_columns = ['Asset_ID','High','Low','Close','Open','VWAP','Volume','Count']
            #df.drop(delete_columns, axis=1, inplace=True)
            for colum in delete_columns:
                del df[colum]; gc.collect();
                
            df.reset_index(drop=True, inplace=True)
            
            ans = df.copy()
        else:
            
            start = time.time()
            ans = pd.DataFrame(index=[0], columns=[])
            
            fe_list = ['VWAP_shift15','VWAP_shift45','VWAP_shift90','VWAP_shift180','VWAP_shift210','VWAP_shift310','VWAP_shift350','VWAP_shift450','ROCP','MOM','RSI','macdhist0','macdhist1','macdhist5','macdhist15','LINEARREG_ANGLE','HT_DCPERIOD','AROONOSC','aroondown','aroonup','PLUS_DI','BBANDS']
            future_list = []
            with futures.ThreadPoolExecutor() as executor:
                for i in range(len(fe_list)):
                    # タスクを追加する。
                    future = executor.submit(self.task, i, df)
                    # Future オブジェクトを記録する。
                    future_list.append(future)
            print ("0.0elapsed_time:{0}".format(time.time() - start) + "[sec]")
            
            start = time.time()
            for i, x in enumerate(future_list):
                ans[fe_list[i]] = x.result()
            print ("0.1elapsed_time:{0}".format(time.time() - start) + "[sec]")
            

            
        start = time.time()
        
        ans.replace([np.inf, -np.inf], np.nan, inplace=True)
        ans.fillna(method='ffill', inplace=True)
        ans.fillna(0, inplace=True)
        
#         dfX = df.copy()
#         dfX.drop(categorical_features, axis=1, inplace=True)
#         columns = dfX.columns.tolist()
#         X = dfX.values
#         if not test:
#             del dfX
#             gc.collect()
        """正規化 sklearn
        scaler = StandardScaler()
        scaler.fit(X)
        X = scaler.transform(X)"""
        
        print ("1.3elapsed_time:{0}".format(time.time() - start) + "[sec]")
        start = time.time()
        
        #最小値の計算
        #print(ans.columns.values.tolist()) 
#         mean_p = [-1.5995870828963125e-06, 0.9999936105140329, 0.9999798805796092, 0.999957909646952, 0.9999094811182778, 0.9998935809324517, 0.9998386623475682, 0.9998166999303114, 0.999761023451013, 2.68893869435963, 50.17742425201508, 0.36609391023010945, -8.513729049383712e-06, -8.536061613701882e-06, -8.62574773048785e-06, -8.817869229881006e-06, 17.394611336872575, 0.8841851233916151, 21.31082820733216, 0.7922342501475305, 2.1344215132277884, 44.66257903897539, 46.7970005522039, 7.528460106582875, 4.479986553198653, -0.05065731832118273]
#         std_p = [0.0020139245973895704, 0.00526436090984338, 0.009028368384249637, 0.012659549928699778, 0.01764081834023001, 0.019017750981543444, 0.022929407656516868, 0.024322060457582173, 0.02745090276263502, 338.29863320746654, 3.292447808247903, 6.531061504548422, 0.08495146561479067, 0.08495145986995727, 0.08495143670699436, 0.0849513938680594, 15.733174955003946, 35.995526034504415, 5.35233708812034, 1.0490572699388196, 55.95956070093342, 33.11596376614276, 33.72480547239823, 4.476566042660567, 1.3812689854022944, 2.2501479214649804]
# #         mean_p = pd.Series(mean_p, index=['Target', 'VWAP_shift15', 'VWAP_shift45', 'VWAP_shift90', 'VWAP_shift180', 'VWAP_shift210', 'VWAP_shift310', 'VWAP_shift350', 'VWAP_shift450', 'MOM', 'RSI', 'CMO', 'macdhist0', 'macdhist1', 'macdhist5', 'macdhist15', 'ADX', 'LINEARREG_ANGLE', 'HT_DCPERIOD', 'NATR', 'AROONOSC', 'aroondown', 'aroonup', 'PLUS_DI', 'BBANDS', 'CDL2CROWS'])
# #         std_p = pd.Series(std_p, index=['Target', 'VWAP_shift15', 'VWAP_shift45', 'VWAP_shift90', 'VWAP_shift180', 'VWAP_shift210', 'VWAP_shift310', 'VWAP_shift350', 'VWAP_shift450', 'MOM', 'RSI', 'CMO', 'macdhist0', 'macdhist1', 'macdhist5', 'macdhist15', 'ADX', 'LINEARREG_ANGLE', 'HT_DCPERIOD', 'NATR', 'AROONOSC', 'aroondown', 'aroonup', 'PLUS_DI', 'BBANDS', 'CDL2CROWS'])
#         mean_p = pd.Series(mean_p, index=ans.columns.values.tolist())
#         std_p = pd.Series(std_p, index=ans.columns.values.tolist())
#         #mean_p = ans.mean()
#         #print(mean_p)
#         #最大値の計算
#         #std_p = ans.std(ddof = 0)
#         #print(std_p.tolist())
#         #標準化の計算
#         ans = (ans - mean_p) / (std_p)

        if save_fet:
            ans.to_csv(save_name+'.csv', index=False)
            
        if save_mem:
            ans = reduce_mem_usage(ans)
        
        if not test:
            del df_list, df
            gc.collect()
        print ("1.4elapsed_time:{0}".format(time.time() - start) + "[sec]")

        return ans
    
    def normalize(self, df):
        return (df - df.mean()) / df.std(ddof=0)

# Util

In [5]:
from sklearn.metrics import accuracy_score
import numpy as np

class Util():

    def __init__(self) -> None:
        pass

    def data_conv(self, data):
        #data = (data > 0.5).astype(int)
        return data


    def accuracy_score(self, train, predict):
        """
        どのくらい答えに近いか評価するスコアを出す
        あっているほど数値が高いようにする
        コンペによって評価方法が違うからこれを変える
        """
        #ピアソンの相関係数
        return np.corrcoef(train,predict)[0,1]

# モデル

In [12]:
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from catboost import Pool
from catboost import CatBoost, CatBoostRegressor, CatBoostClassifier
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import pickle

from memory_profiler import profile
import gc


ut = Util()

class Models:
    def __init__(self,ID) -> None:
        fileID = ID
        self.models_xgboost = []
        for fold_id in range(4):
            try:
                with open('xgboost'+str(fileID)+'0'+str(fold_id)+'.pickle', 'rb') as web:
                    self.models_xgboost.append(pickle.load(web))
                #print('load')
            except FileNotFoundError:
                #print('Not find '+str(fileID)+'0'+str(fold_id))
                pass
    
    def __del__(self):
        """オブジェクトが破棄されるとき呼び出される"""
        print('Models died:', id(self))

    def select_model(self, categorical_features, model_name, learn_type, fileID=0, X_train=None, y_train=None, X_valid=None, y_valid=None, X_test=None):
        if model_name == "random_forest":
            score, y_val_pre, y_pred = self.random_forest(learn_type, fileID=fileID, X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, X_test=X_test)
        elif model_name == "light_gbm":
            score, y_val_pre, y_pred = self.light_gbm(categorical_features, learn_type, fileID=fileID, X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, X_test=X_test)
        elif model_name == 'xgboost':
            score, y_val_pre, y_pred = self.xgboost(learn_type, fileID=fileID, X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, X_test=X_test)
        elif model_name == "catboost":
            score, y_val_pre, y_pred = self.catboost(categorical_features, learn_type, fileID=fileID, X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, X_test=X_test)
        elif model_name == "logistic_regression":
            score, y_val_pre, y_pred = self.logistic_regression(learn_type, fileID=fileID, X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, X_test=X_test)
        elif model_name == "dnn":
            score, y_val_pre, y_pred = self.dnn(learn_type, fileID=fileID, X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, X_test=X_test)
        else:
            raise NameError("指定されたアルゴリズムは存在しません")
            

        return score, y_val_pre, y_pred
    
    def KFold(self, categorical_features, model_name, learn_type, fileID=0, X_train=None, y_train=None, X_test=None, n_splits=4):

        cv_score, oof_pre, y_sub = None, None, None
        scores = []
        oof_pre = np.array([])
        valid_indexs = np.array([])
        y_preds = []
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=0)
        if learn_type=='learn':
            for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
                X_tr = X_train.loc[train_index, :]
                X_val = X_train.loc[valid_index, :]
                y_tr = y_train[train_index]
                y_val = y_train[valid_index]

                score, y_val_pre, _ = self.select_model(categorical_features, model_name, 'learn', fileID=str(fileID)+str(fold_id),X_train=X_tr, y_train=y_tr, X_valid=X_val, y_valid=y_val)

                scores.append(score)
                oof_pre = np.append(oof_pre,y_val_pre)
                valid_indexs = np.append(valid_indexs, valid_index)

            oof_pre = oof_pre[np.argsort(valid_indexs)]
            cv_score = sum(scores) / len(scores)
        elif learn_type=='predict':
            future_list = []
            with futures.ThreadPoolExecutor() as executor:
                for fold_id in range(n_splits):
                    future = executor.submit(self.select_model, categorical_features, model_name, 'predict', fileID=str(fileID)+str(fold_id), X_test=X_test)
                    future_list.append(future)
                    #score, y_val_pre, y_pred = self.select_model(categorical_features, model_name, 'predict', fileID=str(fileID)+str(fold_id), X_test=X_test)
#                     oof_pre = np.append(oof_pre,y_val_pre)
#                     y_preds.append(y_pred)
                    
            for i, x in enumerate(future_list):
                score, y_val_pre, y_pred = x.result()
                oof_pre = np.append(oof_pre,y_val_pre)
                y_preds.append(y_pred)
                    
            
            y_sub = sum(y_preds) / len(y_preds)

        return cv_score, oof_pre, y_sub

    def random_forest(self, learn_type, fileID=0, X_train=None, y_train=None, X_valid=None, y_valid=None, X_test=None, n_estimators=67, max_depth=6, random_state=0):
        """
        pandasでの教師データ
        パラメータ
        return valスコア(float)、その取り出し方での予測値
        """
        print('========random_forest========')
        score, y_val_pre, y_pred = None, None, None
        if learn_type=='predict':
            with open('RandomForest'+str(fileID)+'.pickle', 'rb') as web:
                RandomForest = pickle.load(web)
            y_pred = RandomForest.predict(X_test)
        elif learn_type=='learn':
            #RandomForest = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
            RandomForest = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
            RandomForest.fit(X_train, y_train)
            y_val_pre = RandomForest.predict(X_valid)
            score = ut.accuracy_score(y_valid, y_val_pre)
            with open('RandomForest'+str(fileID)+'.pickle', 'wb') as web:
                pickle.dump(RandomForest , web)

        return score, y_val_pre, y_pred

    def light_gbm(self, categorical_features, learn_type, fileID=0, X_train=None, y_train=None, X_valid=None, y_valid=None, X_test=None, params = {'objective': 'binary','max_bin': 284,'learning_rate': 0.068,'num_leaves': 45}):
        """
        pandasでの教師データ
        categorical_features:カテゴリかる属性のカラム名を示したリスト
        パラメータ
        return valスコア(float), y_val_pre(valでの予測値), その取り出し方での予測値
        """
        print('========light_gbm========')
        score, y_val_pre, y_pred = None, None, None
        if learn_type=='predict':
            with open('light_gbm'+str(fileID)+'.pickle', 'rb') as web:
                model = pickle.load(web)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            y_pred = ut.data_conv(y_pred)
        elif learn_type=='learn':
            lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
            lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)
            model = lgb.train(params, lgb_train,valid_sets=[lgb_train, lgb_eval],verbose_eval=10,num_boost_round=1000,early_stopping_rounds=10)

            y_val_pre = model.predict(X_valid, num_iteration=model.best_iteration)
            y_val_pre = ut.data_conv(y_val_pre)
            score = ut.accuracy_score(y_valid, y_val_pre)
            with open('light_gbm'+str(fileID)+'.pickle', 'wb') as web:
                pickle.dump(model , web)

        return score, y_val_pre, y_pred
    
    def xgboost(self, learn_type, fileID=0, X_train=None, y_train=None, X_valid=None, y_valid=None, X_test=None, params = {'tree_method': 'gpu_hist', 'objective': 'reg:squarederror','silent':1, 'random_state':0,'learning_rate': 0.15, 'eval_metric': 'rmse',}, num_round = 450):
        """
        pandasでの教師データ
        categorical_features:カテゴリかる属性のカラム名を示したリスト
        パラメータ
        return valスコア(float), y_val_pre(valでの予測値), その取り出し方での予測値
        """
        score, y_val_pre, y_pred = None, None, None
        if learn_type=='predict':
            test = xgb.DMatrix(X_test)
            model = self.models_xgboost[int(str(fileID)[-1])]
            y_pred = model.predict(test)
        elif learn_type=='learn':
            print('========xgboost========')
            train = xgb.DMatrix(X_train, label=y_train)
            valid = xgb.DMatrix(X_valid, label=y_valid)
            self.model_xgboost = xgb.train(params,
                    train,#訓練データ
                    num_round,#設定した学習回数
                    early_stopping_rounds=20,
                    evals=[(train, 'train'), (valid, 'eval')],
                    verbose_eval=100
                    )
            y_val_pre = self.model_xgboost.predict(valid)
            score = ut.accuracy_score(y_valid, y_val_pre)
            with open('xgboost'+str(fileID)+'.pickle', 'wb') as web:
                pickle.dump(self.model_xgboost, web)
                
        
        
#         _, ax = plt.subplots(figsize=(12, 15))
#         xgb.plot_importance(self.model_xgboost,
#                     ax=ax,
#                     importance_type='gain',
#                     show_values=False)
#         plt.show()
        
        
        return score, y_val_pre, y_pred

    def catboost(self, categorical_features, learn_type, fileID=0, X_train=None, y_train=None, X_valid=None, y_valid=None, X_test=None, params ={'depth' : 3,'learning_rate' : 0.054,'early_stopping_rounds' : 9,'iterations' : 474, 'loss_function' : 'RMSE', 'random_seed' :0}):
        """
        pandasでの教師データ
        categorical_features:カテゴリかる属性のカラム名を示したリスト
        パラメータ
        return valスコア(float), y_val_pre(valでの予測値), その取り出し方での予測値
        """
        print('========catboost========')
        score, y_val_pre, y_pred = None, None, None
        if learn_type=='predict':
            with open('catboost'+str(fileID)+'.pickle', 'rb') as web:
                model = pickle.load(web)
            y_pred = model.predict(X_test)
            y_pred = ut.data_conv(y_pred)
        elif learn_type=='learn':
            train = Pool(X_train, y_train, cat_features=categorical_features)
            eval = Pool(X_valid, y_valid, cat_features=categorical_features)
            #cab = CatBoostClassifier(custom_loss=['Accuracy'],random_seed=0)
            #cab = CatBoostClassifier(**params)
            cab = CatBoostRegressor(random_seed=0)
            cab = CatBoostRegressor(**params)
            model = cab.fit(train, eval_set=eval)

            y_val_pre = model.predict(X_valid)
            y_val_pre = ut.data_conv(y_val_pre)
            score = ut.accuracy_score(y_valid, y_val_pre)
            with open('catboost'+str(fileID)+'.pickle', 'wb') as web:
                pickle.dump(model , web)

        return score, y_val_pre, y_pred

    def logistic_regression(self, learn_type, fileID=0, X_train=None, y_train=None, X_valid=None, y_valid=None, X_test=None):
        """
        pandasでの教師データ
        パラメータ
        return valスコア(float)、その取り出し方での予測値
        """
        print('========logistic_regression========')
        score, y_val_pre, y_pred = None, None, None
        if learn_type=='predict':
            with open('logistic_regression'+str(fileID)+'.pickle', 'rb') as web:
                model = pickle.load(web)
            y_pred = model.predict(X_test)
            y_pred = ut.data_conv(y_pred)
        elif learn_type=='learn':
            #model = LogisticRegression(penalty='l2', solver='sag', random_state=0)
            model = ElasticNet(random_state=0)
            model.fit(X_train, y_train)
            y_val_pre = model.predict(X_valid)
            y_val_pre = ut.data_conv(y_val_pre)
            score = ut.accuracy_score(y_valid, y_val_pre)
            with open('logistic_regression'+str(fileID)+'.pickle', 'wb') as web:
                pickle.dump(model , web)

        return score, y_val_pre, y_pred

    def dnn(self, learn_type, fileID=0, X_train=None, y_train=None, X_valid=None, y_valid=None, X_test=None):
        
        print('========dnn========')
        score, y_val_pre, y_pred = None, None, None

        lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay( \
                    initial_learning_rate=0.001, #初期の学習率
                    decay_steps=3, #減衰ステップ数
                    decay_rate=0.01, #最終的な減衰率 
                    staircase=True)

        model=Sequential()
        model.add(Dense(len(X_train.columns),input_shape=(len(X_train.columns),),activation='relu',
                    kernel_regularizer=keras.regularizers.l2(0.001), #重みの正則化考慮
                    kernel_initializer='random_uniform',
                    bias_initializer='zero'))
                    
        model.add(BatchNormalization()) #バッチ正規化
        model.add(Dropout(0.1)) # ドロップアウト層・ドロップアウトさせる割合
        model.add(Dense(int(len(pd.DataFrame(X_train).columns)/2),activation='sigmoid'))

        model.add(BatchNormalization()) #バッチ正規化
        model.add(Dropout(0.1)) # ドロップアウト層・ドロップアウトさせる割合
        model.add(Dense(int(len(pd.DataFrame(X_train).columns)/2),activation='sigmoid'))

        model.add(BatchNormalization()) #バッチ正規化
        model.add(Dropout(0.1)) # ドロップアウト層・ドロップアウトさせる割合
        model.add(Dense(len(pd.DataFrame(y_train).columns),activation='sigmoid'))
        Ecall=EarlyStopping(monitor='val_loss',patience=1000,restore_best_weights=False)
        model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=lr_schedule))
        model.summary()

        if learn_type=='predict':
            model.load_weights('dnn'+str(fileID)+'.h5')
            y_pred = model.predict(X_test)
            y_pred = ut.data_conv(y_pred)
        elif learn_type=='learn':
            res=model.fit(X_train.values,y_train.values,epochs=3,callbacks=[Ecall],verbose=1,validation_data=(X_valid.values,y_valid.values))
            y_val_pre = model.predict(X_valid)[:,0]
            y_val_pre = ut.data_conv(y_val_pre)
            print(y_valid.shape)
            print(y_val_pre.shape)
            score = ut.accuracy_score(y_valid, y_val_pre)
            model.save_weights('dnn'+str(fileID)+'.h5')

        return score, y_val_pre, y_pred

# パラメータオプティマイザー

In [13]:
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import log_loss

#md = Models(0)

#random_forest : {'n_estimators': 67, 'max_depth': 6}
#light_gbm : {'max_bin': 284, 'learning_rate': 0.06759289191947715, 'num_leaves': 45}
#xgboost : {'learning_rate': 0.180343853211702, 'num_round': 394}
#catboost : {'depth': 3, 'learning_rate': 0.053925065258405916, 'early_stopping_rounds': 9, 'iterations': 474}
class Optimizer():
    def __init__(self) -> None:
        pass

    def param_opt(self, model_name, X_train, y_train, categorical_features=None):
        """
        パラメータオプティマイザー
        model name list: random_forest, light_gbm
        """
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3,random_state=0, stratify=y_train)
        study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
        if model_name == "random_forest":
            study.optimize(self.objective_random_forest(X_train, y_train, X_valid, y_valid), n_trials=80)
            return study.best_params

        elif model_name == "light_gbm":
            study.optimize(self.objective_light_gbm(X_train, y_train, X_valid, y_valid, categorical_features), n_trials=80)
            return study.best_params

        elif model_name == "xgboost":
            study.optimize(self.objective_xgboost(X_train, y_train, X_valid, y_valid), n_trials=80)
            return study.best_params

        elif model_name == "catboost":
            study.optimize(self.objective_catboost(X_train, y_train, X_valid, y_valid, categorical_features), n_trials=80)
            return study.best_params
            
        elif model_name == 'logistic_regression':
            raise NameError('logistic_regressionはパラメータが存在しないのでサポートしていません')

    def objective_random_forest(self, X_train, y_train, X_valid, y_valid):
        def objective(trial):
            n_estimators = trial.suggest_int('n_estimators', 10, 300)
            max_depth = trial.suggest_int('max_depth', 1, 15)
            _, y_val_pre, _ = md.random_forest('learn', X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, n_estimators=n_estimators, max_depth=max_depth, random_state=0)
            
            score = log_loss(y_valid, y_val_pre)

            return score
        return objective

    def objective_light_gbm(self, X_train, y_train, X_valid, y_valid, categorical_features):
        def objective(trial):
            params = {
            'objective': 'binary',
            'max_bin': trial.suggest_int('max_bin', 255, 500),
            'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
            'num_leaves': trial.suggest_int('num_leaves', 32, 128),
            }
            _, y_val_pre, _ = md.light_gbm(categorical_features, 'learn', X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, params=params)
            
            score = log_loss(y_valid, y_val_pre)

            return score
        return objective

    def objective_xgboost(self, X_train, y_train, X_valid, y_valid):
        def objective(trial):
            params = {'objective': 'reg:squarederror',
                    'silent':1, 
                    'random_state':0,
                    'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.2), 
                    'eval_metric': 'rmse',
            }
            num_round = trial.suggest_int('num_round', 100, 900)
            _, y_val_pre, _ = md.xgboost('learn', X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, params=params, num_round=num_round)
            
            score = log_loss(y_valid, y_val_pre)

            return score
        return objective

    def objective_catboost(self, X_train, y_train, X_valid, y_valid, categorical_features):
        def objective(trial):
            params = {
                'depth' : trial.suggest_int('depth', 1, 15),                  # 木の深さ
                'learning_rate' : trial.suggest_uniform('learning_rate', 0.01, 0.1),       # 学習率
                'early_stopping_rounds' : trial.suggest_int('early_stopping_rounds', 3, 20),
                'iterations' : trial.suggest_int('iterations', 50, 500), 
                'custom_loss' :['Accuracy'], 
                'random_seed' :0
            }
            _, y_val_pre, _ = md.catboost(categorical_features, 'learn', X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid, params=params)
            
            score = log_loss(y_valid, y_val_pre)

            return score
        return objective

    #LogisticRegressionはパラメータがない

# アンサンブル

In [14]:
from sklearn import ensemble
import numpy as np
import pandas as pd

#md = Models()
ut = Util()

class Ensemble():

    def __init__(self) -> None:
        pass

    def stacking(self, categorical_features, learn_type, fileID=0, X_train=None, y_train=None, X_test=None, fst_lay=['random_forest', 'light_gbm', 'xgboost', 'catboost'], snd_lay='light_gbm', enable_2ndorigx=True):
        #enable_2ndorigx:二層目にオリジナルの入力データを入力するか

        stack_oof_pred = []
        stack_pred = []
        for index, model_name in enumerate(fst_lay):
            
            if learn_type=='learn':
                cv_score, oof_pre, y_sub = md.KFold(categorical_features, model_name, learn_type, fileID=fileID+'0', X_train=X_train, y_train=y_train)
                stack_oof_pred = oof_pre if index == 0 else np.c_[stack_oof_pred, oof_pre]
            elif learn_type=='predict':
                cv_score, oof_pre, y_sub = md.KFold(categorical_features, model_name, learn_type, fileID=fileID+'0', X_test=X_test)
                stack_pred = y_sub if index == 0 else np.c_[stack_pred, y_sub]
            else:
                raise NameError("指定されたlearn_typeは存在しません")

        if enable_2ndorigx:
            X_train2 =  pd.concat([pd.DataFrame(stack_oof_pred), X_train], axis=1)
            X_test2 =  pd.concat([pd.DataFrame(stack_pred), X_test], axis=1)
        else:
            X_train2 = pd.DataFrame(stack_oof_pred)
            X_test2 = pd.DataFrame(stack_pred)
            categorical_features = []

        #二層目
        if learn_type=='learn':
            cv_score, oof_pre, y_sub = md.KFold(categorical_features, snd_lay, learn_type, fileID=fileID+'1', X_train=X_train2, y_train=y_train)
        elif learn_type=='predict':
            cv_score, oof_pre, y_sub = md.KFold(categorical_features, snd_lay, learn_type, fileID=fileID+'1', X_test=X_test2)
            y_sub = ut.data_conv(y_sub)

        return cv_score, oof_pre, y_sub

    def mean(self, categorical_features, learn_type, fileID=0, X_train=None, y_train=None, X_test=None, models=['random_forest', 'light_gbm', 'xgboost', 'catboost'], type='mean'):
        '''
        type:平均の取り方 
        mean -> 算術平均
        hmean -> 調和平均
        gmean -> 幾何平均
        '''

        stack_oof_pred = []
        stack_pred = []
        for index, model_name in enumerate(models):
            if learn_type=='learn':
                cv_score, oof_pre, y_sub = md.KFold(categorical_features, model_name, learn_type, fileID=fileID+'0', X_train=X_train, y_train=y_train)
                stack_oof_pred = oof_pre if index == 0 else np.c_[stack_oof_pred, oof_pre]
            elif learn_type=='predict':
                cv_score, oof_pre, y_sub = md.KFold(categorical_features, model_name, learn_type, fileID=fileID+'0', X_test=X_test)
                stack_pred = y_sub if index == 0 else np.c_[stack_pred, y_sub]
            else:
                raise NameError("指定されたlearn_typeは存在しません")

        if type == 'mean':
            y_off = np.average(stack_oof_pred, axis=1)
            y_sub = np.average(stack_pred, axis=1)
        elif type == 'hmean':
            from scipy.stats import hmean
            y_off = hmean(stack_oof_pred, axis = 1)
            y_sub = hmean(stack_pred, axis = 1)
        elif type == 'gmean':
            from scipy.stats.mstats import gmean
            y_off = gmean(stack_oof_pred, axis = 1)
            y_sub = gmean(stack_pred, axis = 1)
        
        y_off = ut.data_conv(y_off)
        y_sub = ut.data_conv(y_sub)

        cv_score = ut.accuracy_score(y_train, y_off)
        
        return cv_score, oof_pre, y_sub

# コントローラー

In [15]:
import numpy as np
import pandas as pd

op = Optimizer()
ens = Ensemble()

class Controller():

    def __init__(self,ID) -> None:
        self.md = Models(ID)
        self.ID = str(ID)

    def opt(self, X_train, y_train):
        print(op.param_opt('light_gbm', X_train, y_train))

    def KFold_learn(self, categorical_features, X_train, y_train, model_name):
        cv_score, y_val_pre, y_sub = self.md.KFold(categorical_features, model_name, 'learn', fileID=self.ID+'0', X_train=X_train, y_train=y_train)

        print('CV score-----------------------------------',cv_score)
        #random_forest 0.822635113928818
        #light_gbm 0.8293829640323895
        #catboost 0.8204004770573097
        #logistic_regression 0.6846023476241291
        #xgboost 0.8192643274119641
        #dnn 0.8159060950348378s
        
        return cv_score

    def KFold_predict(self, categorical_features, X_test, model_name):
        cv_score, y_val_pre, y_sub = self.md.KFold(categorical_features, model_name, 'predict', fileID=self.ID+'0', X_test=X_test)
        
        return y_sub

    def stacking_learn(self, categorical_features, X_train, y_train, fst_lay=['random_forest', 'light_gbm', 'xgboost', 'catboost'], snd_lay='light_gbm', enable_2ndorigx=False):
        cv_score, _, y_sub = ens.stacking(categorical_features, 'learn', fileID=self.ID, X_train=X_train, y_train=y_train, fst_lay=fst_lay, snd_lay=snd_lay, enable_2ndorigx=enable_2ndorigx)

        print('CV score-----------------------------------',cv_score)
        #0.8293829640323895 2ndlgtm
        #0.8237712635741635 2ndrandomforest

    def stacking_predict(self, categorical_features, X_test, fst_lay=['random_forest', 'light_gbm', 'xgboost', 'catboost'], snd_lay='light_gbm', enable_2ndorigx=False):
        _, _, y_sub = ens.stacking(categorical_features, 'predict', fileID=self.ID, X_test=X_test, fst_lay=fst_lay, snd_lay=snd_lay, enable_2ndorigx=enable_2ndorigx)

        sub = pd.read_csv('input/titanic/gender_submission.csv')
        sub['Survived'] = y_sub
        sub.to_csv('submission.csv', index=False)
        #0.8293829640323895

    def mean_learn(self, categorical_features, learn_type, X_train, y_train, models=['random_forest', 'light_gbm', 'xgboost', 'catboost'], type='mean'):
        cv_score, _, y_sub = ens.mean(categorical_features, learn_type, fileID=self.ID, X_train=X_train, y_train=y_train, models=models, type=type)
        #mean:0.8237934904601572 hmean:0.819304152637486 gmean:0.819304152637486
        print('CV score-----------------------------------',cv_score)

    def mean_predict(self, categorical_features, learn_type, X_test, models=['random_forest', 'light_gbm', 'xgboost', 'catboost'], type='mean'):
        cv_score, _, y_sub = ens.mean(categorical_features, learn_type, fileID=self.ID, X_test=X_test, models=models, type=type)

        sub = pd.read_csv('input/titanic/gender_submission.csv')
        sub['Survived'] = y_sub
        sub.to_csv('submission.csv', index=False)

# Main

教師データ読み込み

In [10]:
import numpy as np
import pandas as pd
import random
import os

def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything

data_folder = "../input/g-research-crypto-forecasting/"
#crypto_df = reduce_mem_usage(pd.read_csv(data_folder + 'train.csv'))
crypto_df = pd.read_csv(data_folder + 'train.csv')

train_list = list(range(13))
for Asset_ID in range(13):#通貨別にデータを作りそれを通貨別でリストにDFを格納
    #train = reduce_mem_usage(crypto_df[crypto_df["Asset_ID"]==Asset_ID].set_index("timestamp"))
    train = crypto_df[crypto_df["Asset_ID"]==Asset_ID].set_index("timestamp")
    train_list[Asset_ID] = train
del crypto_df
del train
gc.collect()

22

## 学習

In [33]:
scores = []
fe = Feature()
for Asset_ID in range(13):#通貨別にデータを作りそれを通貨別でリストにDFを格納
    
    print('通貨番号',Asset_ID)
    
    train_raw = train_list[Asset_ID].copy()
    y_train = train_raw['Target'].copy()
    X_train = train_raw.drop('Target', axis=1).copy()
    
    y_train.reset_index(drop=True, inplace=True)
    y_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_train.fillna(method='ffill', inplace=True)
    y_train.fillna(0, inplace=True)
    X_train = fe.conv_data(X_train, Asset_ID, train_list, save_fet=False, save_name='feature')
    #print(train)
    #print(train_list[1])
    categorical_features = ['CDL2CROWS']
    
    ct = Controller(Asset_ID)
            
    cv_score = ct.KFold_learn(categorical_features, X_train, y_train, 'xgboost')
    scores.append(cv_score)
    #ct.stacking_learn(categorical_features, X_train, y_train, fst_lay=['dnn', 'light_gbm', 'xgboost'], snd_lay='light_gbm', enable_2ndorigx=False)
    
    del train_raw, y_train, X_train, ct
    gc.collect()
    
    print('5',psutil.virtual_memory().percent)
    
    
print('CV mean-----------------------------'+str(sum(scores)/len(scores)))

#CV score----------------------------------- 0.23738583123946141 light_gbm 0.2578
#xgboost 0.44045052797107087
#random forest 微妙　0.15 
#catboost 0.06

通貨番号 0
Feature died: 140114302901392
1.3elapsed_time:0.9051985740661621[sec]
Mem. usage decreased to 185.26 Mb (75.0% reduction)
1.4elapsed_time:8.327182054519653[sec]
========xgboost========
[06:27:08] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:0.42501	eval-rmse:0.42501
[100]	train-rmse:0.00470	eval-rmse:0.00482
[200]	train-rmse:0.00443	eval-rmse:0.00459
[300]	train-rmse:0.00424	eval-rmse:0.00444
[400]	train-rmse:0.00408	eval-rmse:0.00431
[449]	train-rmse:0.00402	eval-rmse:0.00426
========xgboost========
[06:27:19] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindi

KeyboardInterrupt: 

In [27]:
#Measure a time(fe)
fe = Feature()
df = train_list[1].copy()
start = time.time()
#df = fe.conv_data(df, 1, train_list, save_fet=False, save_name='feature', save_mem=False)
df = fe.conv_data(df.iloc[-610:,:], 1, train_list, save_fet=False, save_name='feature', save_mem=False, test=True)
print ("0elapsed_time:{0}".format(time.time() - start) + "[sec]")
display(df.iloc[-1,:],'ans')
display(df.info,'ans')

Feature died: 140114321170768
0.0elapsed_time:0.023078203201293945[sec]
0.1elapsed_time:0.009051322937011719[sec]
1.3elapsed_time:0.0012247562408447266[sec]
1.4elapsed_time:1.1920928955078125e-06[sec]
0elapsed_time:0.034983158111572266[sec]


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


VWAP_shift15         0.992272
VWAP_shift45         1.005135
VWAP_shift90         1.017850
VWAP_shift180        1.015129
VWAP_shift210        1.015822
VWAP_shift310        1.013320
VWAP_shift350        1.021136
VWAP_shift450        1.023971
ROCP                -0.012434
MOM               -580.377500
RSI                 46.443191
macdhist0            0.043689
macdhist1            0.044011
macdhist5            0.043150
macdhist15           0.033535
LINEARREG_ANGLE    -80.578859
HT_DCPERIOD         18.429952
AROONOSC           -30.386740
aroondown           84.530387
aroonup             54.143646
PLUS_DI             17.100123
BBANDS               7.386793
Name: 0, dtype: float64

'ans'

<bound method DataFrame.info of    VWAP_shift15  VWAP_shift45  VWAP_shift90  VWAP_shift180  VWAP_shift210  \
0      0.992272      1.005135       1.01785       1.015129       1.015822   

   VWAP_shift310  VWAP_shift350  VWAP_shift450      ROCP       MOM  ...  \
0        1.01332       1.021136       1.023971 -0.012434 -580.3775  ...   

   macdhist1  macdhist5  macdhist15  LINEARREG_ANGLE  HT_DCPERIOD  AROONOSC  \
0   0.044011    0.04315    0.033535       -80.578859    18.429952 -30.38674   

   aroondown    aroonup    PLUS_DI    BBANDS  
0  84.530387  54.143646  17.100123  7.386793  

[1 rows x 22 columns]>

'ans'

In [ ]:
# #del train_raw
# del X_train, y_train
# gc.collect()

In [ ]:
# del train_list
# gc.collect()

In [ ]:
# import sys

# print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
# print(" ------------------------------------ ")
# for var_name in dir():
#     if not var_name.startswith("_"):
#         print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [ ]:
#train_list[0].reset_index(drop=True)

In [ ]:
#fe.conv_data(train_list[0], save_fet=False, save_name='feature')

## predict

In [ ]:
supplemental_df = pd.read_csv(data_folder + 'supplemental_train.csv')

In [ ]:
test_list = list(range(13))
for Asset_ID in range(13):#通貨別にデータを作りそれを通貨別でリストにDFを格納
    supplemental_train = supplemental_df[supplemental_df["Asset_ID"]==Asset_ID].set_index("timestamp")
    if len(supplemental_train) > 1000:#30000
        #supplemental_df.drop(supplemental_df.index[-26000:], inplace=True)
        supplemental_train = supplemental_train.iloc[-1000:,:]
    test_list[Asset_ID] = supplemental_train

In [ ]:
import gresearch_crypto
#以下二つは1セッションで一度しか実行できない
env = gresearch_crypto.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
import time
fe = Feature()
ctlist = []
categorical_features = ['CDL2CROWS']
for Asset_ID in range(13):
    ct = Controller(Asset_ID)
    ctlist.append(ct)
for i, (df_test, df_pred) in enumerate(iter_test):
    start1 = time.time()
    start = time.time()
    for Asset_ID in range(13):
        test_raw = df_test[df_test["Asset_ID"]==Asset_ID].set_index("timestamp")
        test = pd.concat([test_list[Asset_ID], test_raw], sort=False)
        if len(test) > 1000:#30000
            #test.drop(test.index[-26000:], inplace=True)
            test = test.iloc[-1000:,:]
        test_list[Asset_ID] = test.copy()
    print ("1elapsed_time:{0}".format(time.time() - start) + "[sec]")
    for Asset_ID in range(13):
        start = time.time()
        print('通貨番号',Asset_ID)

        test_raw = test_list[Asset_ID].copy()
        
        #print(test_raw)
        
        row_id = test_raw.iat[-1, 9]
        print('row_id :',row_id)
        X_test = test_raw.drop(['Target','row_id'], axis=1).copy()
        print ("2elapsed_time:{0}".format(time.time() - start) + "[sec]")
        start = time.time()
        X_test = fe.conv_data(X_test, Asset_ID, test_list, save_fet=False, save_name='feature', save_mem=False, test=True)
        print ("3elapsed_time:{0}".format(time.time() - start) + "[sec]")
        start = time.time()
        ct = ctlist[Asset_ID]
        y_sub = ct.KFold_predict(categorical_features, X_test, 'xgboost')[-1]
        #y_sub = ct.stacking_predict(categorical_features, X_test)
        print ("4elapsed_time:{0}".format(time.time() - start) + "[sec]")
        
        print('pred :',y_sub)
        #print(df_pred)
        #print(df_pred['row_id'] == row_id)
        
        df_pred.loc[df_pred['row_id'] == row_id, 'Target'] = y_sub
    
    #print(df_pred)
    print ("elapsed_time:{0}".format(time.time() - start1) + "[sec]")
    env.predict(df_pred)   # register your predictions

In [ ]:
fe = Feature()
X_test = test_raw.drop('Target', axis=1).copy()

print(1)
X_test = fe.conv_data(X_test, Asset_ID, test_list, save_fet=False, save_name='feature', save_mem=False).iloc[-1:]
print(2)
y_sub = ct.KFold_predict(categorical_features, X_test, 'xgboost')
print(2)

In [ ]:
y_sub[-1]

# G-Research

* tutrial - [https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition](http://)

# description

* 15分後の価格の変動率を予測する
* 常に市場の傾向が変動するので、確実な予測モデルを立てるのが難しい(傾向が非定常的)
* オーバーフィッティングの可能性が高い
* 通貨間での価格変動の関連性がある。特にビットコインはほかの通貨に影響を与えやすい。
* 将来、価格がどのように動くかを予測することである。過去の価格の時系列データを学習データとして、価格が上がるか下がるか、またどの程度上がるか、すなわち資産リターンを予測

### Data

#### train.csv

* timestamp: データのUNIX秒。すべて60秒間隔よって、一分ごとにデータが与えられている。
* Asset_ID: asset_details.csvに書いている通貨IDと結びついており、通貨の種類の識別に使う。 (e.g. Asset_ID = 1 for Bitcoin)
* Count: 前の一分間で取引された回数
* Open: 始値 (in USD).
* High: 前の一分間での最大の価格 (in USD).
* Low: 前の一分間での最小の価格 (in USD).
* Close: Close price 終値 (in USD).
* Volume: 前の一分間の引通貨量(USD)
* VWAP: 一定期間内での取引価格の、取引量による加重平均
* Target: 15分前の価格との差をlogでとったもの Residual log-returns for the asset over a 15 minute horizon.

#### asset_details.csv

* Asset_ID 通貨ID
* Weight 性能評価するときにその通貨の正答率がどのくらい加味されるかの重み
* Asset_Name IDに結びついている通貨名


### 評価方法





# task

1. 概要(overview)をしっかり読む
2. 似ている過去のコンペを探し、参加し基本的な分析を行う
3. 似たような大会のsolutionを読む
4. 論文を読んでその分野の進捗を見逃さないようにする
5. データを分析し安定したCVのモデルを構築する
6. データ前処理、特徴量エンジニアリングを行い一定のモデルでCVを比較しいい特徴量エンジニアリングを探す
7. モデルの予測と教師データを比較し分析、予測の難しいデータに対し考察
8. 分析に基づき高性能なモデルをアンサンブルなどを取り入れて構築
9.  データ解析、結果分析からより高度な予測の難しいサンプルを解決するモデルを設計
10. 必要であれば前のステップに戻る

# scores

# else

* supplemental_trainとtrainをくっつけて最後に学習
* B (billion)	1,000,000,000

# 可視化

In [ ]:
# import pandas as pd
# import numpy as np

In [ ]:
# data_folder = "../input/g-research-crypto-forecasting/"

# crypto_df = reduce_mem_usage(pd.read_csv(data_folder + 'train.csv'))

In [ ]:
# train_list = []
# for Asset_ID in range(13):#通貨別にデータを作りそれを通貨別でリストにDFを格納
#     train = crypto_df[crypto_df["Asset_ID"]==Asset_ID].set_index("timestamp")
#     train = fe.conv_data(train, save_fet=False, save_name='feature')
#     train_list.append(train)

In [ ]:
# fe = Feature()
# for Asset_ID in range(13):#通貨別にデータを作りそれを通貨別でリストにDFを格納
#     train = train_list[Asset_ID]
    
#     categorical_features = ['Embarked', 'Pclass', 'Sex']
#     y_train = train['Target']
#     X_train = train.drop('Target', axis=1)
    
#     #ct = Controller(Asset_ID)
#     #ct.stacking_learn(categorical_features, X_train, y_train)

In [ ]:
# import gresearch_crypto
# #以下二つは1セッションで一度しか実行できない
# env = gresearch_crypto.make_env()   # initialize the environment
# iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
# for (test_df, sample_prediction_df) in iter_test:
    
#     for Asset_ID in df_test.Asset_ID.unique():
#         df_test = df_test[df_test["Asset_ID"]==Asset_ID].set_index("timestamp")
        
#         test = fe.conv_data(test, save_fet=False, save_name='feature')

#         categorical_features = ['Embarked', 'Pclass', 'Sex']
#         y_train = train['Target']
        
#         ct = Controller(Asset_ID)
#         ct.stacking_predict(categorical_features, X_test)
    
#     #testと教師データを結合して特徴量を作る
    
#     sample_prediction_df['Target'] = 0  # make your predictions here
#     env.predict(sample_prediction_df)   # register your predictions

In [ ]:
# for (test_df, sample_prediction_df) in iter_test:
#     sample_prediction_df['Target'] = 0
#     env.predict(sample_prediction_df)
#     print(test_df)

In [ ]:
# data_folder = "../input/g-research-crypto-forecasting/"
# crypto_df = pd.read_csv(data_folder + 'supplemental_train.csv')

In [ ]:
# import pandas_profiling

# train_list[0].profile_report()

# feature optimizer

In [ ]:
# import numpy as np
# import pandas as pd
# data_folder = "../input/g-research-crypto-forecasting/"
# crypto_df = pd.read_csv(data_folder + 'train.csv')

In [ ]:
# import optuna
# import talib

# price = np.array(crypto_df['Close'])
# returns = np.array(crypto_df['Close'].shift(-15)) - price
# #returns = np.array(crypto_df['Target'])

# #ROCP {'timeperiod': 6} best param -0.0005372071418046449 best score
# #MOM {'timeperiod': 127} best param -0.001971999282000135 best score
# #RSI -0.0009095762807234701 3
# #EMA {'timeperiod': 11} best param -0.00034526380519110374 best score

# def objective(trial):
#     timeperiod = trial.suggest_int('timeperiod', 2, 240)
    
#     df = talib.ROCP(price, timeperiod=timeperiod)
#     returns_new = returns[~np.isnan(df)]
#     df_new = df[~np.isnan(df)]
#     returns_last = returns_new[~np.isnan(returns_new)]
#     df_new = df_new[~np.isnan(returns_new)]
#     ic = get_ic(df_new, returns_last)
#     print(ic, timeperiod)
#     return -abs(ic)

# def get_ic(x, returns, normalize=True) -> float:
#     """
#     :param np.ndarray x: 指標
#     :param np.ndarray returns: リターン
#     :param bool normalize: x をスケーリングするかどうか
#     """
#     assert(len(x) == len(returns))
#     x = (x - x.mean()) / x.std() if normalize else x
#     returns = (returns - returns.mean()) / returns.std() if normalize else returns
#     ic = np.corrcoef(x, returns)[0, 1]

#     return ic

# study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
# study.optimize(objective, n_trials=100)
# print(study.best_params,'best param')
# print(study.best_value,'best score')